In [1]:
import sqlite3
import csv
from pathlib import Path
from datetime import datetime

CSV_PATH = Path("combined_weather.csv")
DB_PATH = Path("weather.db")

# 1) Define schema with explicit types
DDL = """
PRAGMA journal_mode = WAL;
PRAGMA synchronous = NORMAL;

CREATE TABLE IF NOT EXISTS weather_obs (
    id INTEGER PRIMARY KEY,
    name TEXT,                       -- e.g., "34.68,32.61" (lat,lon or station name)
    lat REAL,                        -- parsed from name if it looks like "lat,lon"
    lon REAL,                        -- parsed from name if it looks like "lat,lon"
    datetime_utc TEXT NOT NULL,      -- ISO8601 UTC timestamp
    temp REAL,
    feelslike REAL,
    dew REAL,
    humidity REAL,
    precip REAL,
    precipprob REAL,
    preciptype TEXT,
    snow REAL,
    snowdepth REAL,
    windgust REAL,
    windspeed REAL,
    winddir REAL,
    sealevelpressure REAL,
    cloudcover REAL,
    visibility REAL,
    solarradiation REAL,
    solarenergy REAL,
    uvindex REAL,
    severerisk REAL,
    conditions TEXT,
    icon TEXT,
    stations_raw TEXT                -- original comma-separated station codes
);

CREATE TABLE IF NOT EXISTS weather_obs_stations (
    obs_id INTEGER,
    station_code TEXT,
    FOREIGN KEY(obs_id) REFERENCES weather_obs(id)
);

CREATE INDEX IF NOT EXISTS idx_weather_obs_dt ON weather_obs(datetime_utc);
CREATE INDEX IF NOT EXISTS idx_weather_obs_latlon ON weather_obs(lat, lon);
CREATE INDEX IF NOT EXISTS idx_weather_obs_name ON weather_obs(name);
CREATE INDEX IF NOT EXISTS idx_weather_obs_conditions ON weather_obs(conditions);
CREATE INDEX IF NOT EXISTS idx_weather_obs_station ON weather_obs_stations(station_code);
"""

def parse_lat_lon_from_name(name):
    # If name looks like "lat,lon" return floats, else (None, None)
    try:
        parts = [p.strip() for p in name.split(",")]
        if len(parts) == 2:
            lat = float(parts[0])
            lon = float(parts[1])
            return lat, lon
    except Exception:
        pass
    return None, None

def normalize_float(s):
    if s is None or s == "" or s.lower() == "null":
        return None
    try:
        return float(s)
    except ValueError:
        return None

def normalize_str(s):
    if s is None:
        return None
    s = s.strip()
    return s if s != "" else None

def normalize_datetime(s):
    # Input like "2024-07-25T00:00:00" -> keep as ISO8601
    # Optionally ensure 'Z' for UTC:
    s = s.strip()
    # Validate parse; if fails, keep raw
    try:
        dt = datetime.fromisoformat(s.replace("Z",""))
        # Store as UTC ISO without timezone notation (or append 'Z' if preferred)
        return dt.strftime("%Y-%m-%dT%H:%M:%S")
    except Exception:
        return s

def split_stations(stations_field):
    if stations_field is None:
        return []
    s = stations_field.strip().strip('"')
    if not s:
        return []
    # stations are comma-separated codes, keep each stripped
    return [t.strip() for t in s.split(",") if t.strip()]

def main():
    if not CSV_PATH.exists():
        raise FileNotFoundError(f"{CSV_PATH} not found")

    conn = sqlite3.connect(DB_PATH)
    conn.executescript(DDL)

    with conn, open(CSV_PATH, newline="", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        insert_obs = conn.cursor()
        insert_station = conn.cursor()

        for row in reader:
            name = normalize_str(row.get("name"))
            lat, lon = parse_lat_lon_from_name(name) if name else (None, None)

            dt = normalize_datetime(row.get("datetime",""))

            values = (
                name,
                lat, lon,
                dt,
                normalize_float(row.get("temp")),
                normalize_float(row.get("feelslike")),
                normalize_float(row.get("dew")),
                normalize_float(row.get("humidity")),
                normalize_float(row.get("precip")),
                normalize_float(row.get("precipprob")),
                normalize_str(row.get("preciptype")),
                normalize_float(row.get("snow")),
                normalize_float(row.get("snowdepth")),
                normalize_float(row.get("windgust")),
                normalize_float(row.get("windspeed")),
                normalize_float(row.get("winddir")),
                normalize_float(row.get("sealevelpressure")),
                normalize_float(row.get("cloudcover")),
                normalize_float(row.get("visibility")),
                normalize_float(row.get("solarradiation")),
                normalize_float(row.get("solarenergy")),
                normalize_float(row.get("uvindex")),
                normalize_float(row.get("severerisk")),
                normalize_str(row.get("conditions")),
                normalize_str(row.get("icon")),
                normalize_str(row.get("stations")),
            )

            insert_obs.execute("""
                INSERT INTO weather_obs
                (name, lat, lon, datetime_utc, temp, feelslike, dew, humidity,
                 precip, precipprob, preciptype, snow, snowdepth, windgust, windspeed,
                 winddir, sealevelpressure, cloudcover, visibility, solarradiation,
                 solarenergy, uvindex, severerisk, conditions, icon, stations_raw)
                VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
            """, values)

            obs_id = insert_obs.lastrowid
            # explode stations to a child table for easier filtering
            for code in split_stations(row.get("stations")):
                insert_station.execute(
                    "INSERT INTO weather_obs_stations (obs_id, station_code) VALUES (?, ?)",
                    (obs_id, code)
                )

    # Optional: vacuum and stats
    with conn:
        conn.execute("ANALYZE;")
    conn.close()
    print(f"Loaded {CSV_PATH} into {DB_PATH}")

if __name__ == "__main__":
    main()


Loaded combined_weather.csv into weather.db
